# Brier Score Calculator

This notebook calculates the Brier Score using the function from sklearn.

In [1]:
!hostname
# Must be used on regular nodes with a lot of ram

c302-006.ls6.tacc.utexas.edu


In [2]:
import pandas as pd
from joblib import Parallel, delayed
from sklearn.metrics import brier_score_loss
from xskillscore import brier_score
import os

WORK = os.environ.get('WORK')
SCRATCH = os.environ.get('SCRATCH')

In [3]:
import glob
era_paths = {month:sorted(list(glob.glob(f'{SCRATCH}/hindcast/era3/*-{str(month).zfill(2)}.parquet',recursive=True))) for month in range(1,13)}
hc_paths = {month:sorted(list(glob.glob(f'{WORK}/hindcast/stage1/surface_variables_*-{str(month).zfill(2)}-*.parquet',recursive=True))) for month in range(1,13)}

In [ ]:
# idx = pd.IndexSlice

# def process_lead_day(era_df,hc_df,lead_day,xskill=False):
#     hc_df = hc_df.loc[idx[:,:,:,lead_day]]
#     era_df = era_df.loc[hc_df.index]

#     if not xskill:
#         return {lead_day:{'BS 25ref':brier_score_loss(era_df['>25mm'],era_df['>25mm climate']),'BS 2.5ref':brier_score_loss(era_df['>2.5mm'],era_df['>2.5mm climate']),
#                       'BS 25':brier_score_loss(era_df['>25mm'],hc_df['>25mm']),'BS 2.5':brier_score_loss(era_df['>2.5mm'],hc_df['>2.5mm']),
#                       }}
#     else:
#         return {lead_day:{
#                         'BS 25ref xskill':brier_score(era_df['>25mm'],era_df['>25mm climate']),'BS 2.5ref':brier_score(era_df['>2.5mm'],era_df['>2.5mm climate']),
#                         'BS 25 xskill':brier_score(era_df['>25mm'],hc_df['>25mm']),'BS 2.5':brier_score(era_df['>2.5mm'],hc_df['>2.5mm']),
#                     }}

# def calc_prob(frame,col):
#     # Both ERA and hindcast use units of meters for precipitation.
#     rain_mm = frame[col]*1000
#     frame['>2.5mm'] = rain_mm>2.5
#     frame['>25mm'] = rain_mm>25
#     return frame

# def process_month(month,xskill=False):
#     with Parallel(n_jobs=-1,prefer="threads",return_as='generator_unordered') as parallel:
#         era_df = pd.concat(parallel(delayed(pd.read_parquet)(file) for file in era_paths[month])).pipe(calc_prob,'total_precipitation')
#         hc_df = pd.concat(parallel(delayed(pd.read_parquet)(file) for file in hc_paths[month])).pipe(calc_prob,'precipitation tp06 [m]')

#     print("load done")
#     # Climatology
#     era_df['month'] = era_df.index.get_level_values('time').month
#     era_df['day'] = era_df.index.get_level_values('time').day
#     era_df['>2.5mm climate'] = era_df.groupby(['month','day'])['>2.5mm'].transform('mean')
#     era_df['>25mm climate'] = era_df.groupby(['month','day'])['>25mm'].transform('mean')
#     era_df = era_df.drop(['month','day'],axis=1)

#     print("start tasks")
#     lead_days = hc_df.index.get_level_values('lead_time').unique()
#     lead_days = lead_days[lead_days<=pd.Timedelta('7D')]

#     results = Parallel(n_jobs=3,return_as='generator_unordered')(delayed(process_lead_day)(era_df,hc_df,lead_day,xskill=xskill) for lead_day in lead_days)
#     return {month:{k:v for item in results for k,v in item.items()}}
#     # return era_df,hc_df,lead_days

# def process_all(xskill=False):
#     return {k:v for list_item in (process_month(month,xskill=xskill) for month in range(1,13)) for k,v in list_item.items()}

In [ ]:
from rainproj.brier_score import process_all
test2 = process_all(era_paths=era_paths,hc_paths=hc_paths,xskill=False)

load done
start tasks


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame.from_dict({(month,lead):metrics for month,lead_list in test2.items() for lead,metrics in lead_list.items()},orient='index').sort_index()
df.index.set_names(['Month','Lead Day'],inplace=True)
df.to_parquet(f'{WORK}/rainproj/data/br_loss_score.parquet')

## Unused Code
***

In [ ]:
raise ValueError

# AORC Addendum

***

In [ ]:
hc_df

precipitation tp06 [m]  >2.5mm  >25mm
date       lat  lon   lead_time                                       
1979-01-01 25.0 235.0 0 days                   0.000096   False  False
                      1 days                   0.004315    True  False
                      2 days                   0.004706    True  False
                      3 days                   0.003211    True  False
                      4 days                   0.001623   False  False
...                                                 ...     ...    ...
2024-01-31 49.5 293.0 3 days                   0.001326   False  False
                      4 days                   0.000955   False  False
                      5 days                   0.009829    True  False
                      6 days                   0.001459   False  False
                      7 days                   0.000030   False  False

[263148336 rows x 3 columns]

In [ ]:
era_df.loc[hc_df.loc[idx[:,:,:,pd.Timedelta('7D')]].index]

total_precipitation  >2.5mm  >25mm
time       lat  lon                                       
1979-01-01 25.0 235.00         1.159403e-04   False  False
                235.25         4.931912e-05   False  False
                235.50         2.076663e-05   False  False
                235.75         4.328787e-06   False  False
                236.00         8.661300e-07   False  False
...                                     ...     ...    ...
2024-01-31 49.5 292.00         2.423152e-04   False  False
                292.25         2.007447e-04   False  False
                292.50         1.713447e-04   False  False
                292.75         1.632310e-04   False  False
                293.00         1.571514e-04   False  False

[32893542 rows x 3 columns]

In [ ]:
idx = pd.IndexSlice

hc_df.loc[idx[:,:,:,pd.Timedelta('7D')]].index

precipitation tp06 [m]  >2.5mm  >25mm
date       lat  lon                                          
1979-01-01 25.0 235.00                0.000064   False  False
                235.25                0.000072   False  False
                235.50                0.000075   False  False
                235.75                0.000079   False  False
                236.00                0.000072   False  False
...                                        ...     ...    ...
2024-01-31 49.5 292.00               -0.000054   False  False
                292.25               -0.000054   False  False
                292.50               -0.000016   False  False
                292.75                0.000022   False  False
                293.00                0.000030   False  False

[32893542 rows x 3 columns]

In [ ]:
common_idx = hc_df.index.droplevel('lead_time').intersection(era_df.index)

In [ ]:
# lead='0D'

# era_1980 = {mo:pd.read_parquet(era_paths[mo]) for mo in range(1,13)}
# hc_1980_0L = {mo:pd.read_parquet(hc_paths[mo]).loc[idx[:,:,:,pd.Timedelta(lead)]] for mo in range(1,13)}

In [ ]:
lead_days[lead_days<=pd.Timedelta('6D')]

TimedeltaIndex(['0 days', '1 days', '2 days', '3 days', '4 days', '5 days',
                '6 days'],
               dtype='timedelta64[ns]', name='lead_time', freq=None)

In [ ]:
hc_df.index.get_level_values('lead_time').max()

Timedelta('7 days 00:00:00')

In [ ]:
idx = pd.IndexSlice

hc_df.loc[idx[:,:,:,pd.Timedelta('7D')]]

precipitation tp06 [m]  >2.5mm  >25mm
date       lat  lon                                          
1979-01-01 25.0 235.00                0.000064   False  False
                235.25                0.000072   False  False
                235.50                0.000075   False  False
                235.75                0.000079   False  False
                236.00                0.000072   False  False
...                                        ...     ...    ...
2024-01-31 49.5 292.00               -0.000054   False  False
                292.25               -0.000054   False  False
                292.50               -0.000016   False  False
                292.75                0.000022   False  False
                293.00                0.000030   False  False

[32893542 rows x 3 columns]

In [ ]:
era_df

total_precipitation  >2.5mm  >25mm
time       lat  lon                                       
1979-01-01 25.0 235.00         1.159403e-04   False  False
                235.25         4.931912e-05   False  False
                235.50         2.076663e-05   False  False
                235.75         4.328787e-06   False  False
                236.00         8.661300e-07   False  False
...                                     ...     ...    ...
2025-01-01 49.5 292.00         3.803242e-04   False  False
                292.25         4.865769e-04   False  False
                292.50         5.792510e-04   False  False
                292.75         5.113427e-04   False  False
                293.00         3.411751e-04   False  False

[32916609 rows x 3 columns]

In [ ]:
brier_score_loss(process(era_1980[1],'total_precipitation')['>25mm'],process(hc_1980_0L[1],'precipitation tp06 [m]')['>25mm'])

In [ ]:
lead='0D'

era_1980 = {mo:pd.read_parquet(era_paths[mo]) for mo in range(1,13)}
hc_1980_0L = {mo:pd.read_parquet(hc_paths[mo]).loc[idx[:,:,:,pd.Timedelta(lead)]] for mo in range(1,13)}

In [ ]:
for mo in range(1,13):
    print(brier_score_loss(process(era_1980[mo],'total_precipitation')['>2.5mm'],process(hc_1980_0L[mo],'precipitation tp06 [m]')['>2.5mm']))

0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516


In [ ]:
brier_score_loss(process(era_1980[1],'total_precipitation')['>25mm'],process(hc_1980_0L[1],'precipitation tp06 [m]')['>25mm'])

0.009891242481578907

In [ ]:
def run_month(lead='0D'):
    losses2p5 = []
    losses25 = []
    for mo in range(1,13):
        era = era_1980[mo]; hc = hc_1980_0L[mo]
        era['mm'] = era['total_precipitation']*1000
        era['>2.5mm'] = era['mm']>2.5
        era['>25mm'] = era['mm']>25

        hc['mm'] = hc['precipitation tp06 [m]']*1000
        hc['>2.5mm'] = hc['mm']>2.5
        hc['>25mm'] = hc['mm']>25
        display(hc['>2.5mm'].values)


        losses2p5 += brier_score_loss(era['>2.5mm'].values,hc['>2.5mm'].values)
        losses25 += brier_score_loss(era['>25mm'].values,hc['>25mm'].values)
        print(losses25)
        print(losses2p5)
    return losses2p5,losses25


In [ ]:
run_month(lead='0D')


array([False, False, False, ..., False, False, False], shape=(715077,))

TypeError: 'float' object is not iterable

In [ ]:
run_month(lead='0D')

total_precipitation        mm  >2.5mm  >25mm
date       lat  lon                                                 
1980-01-01 25.0 235.00             0.000036  0.036219   False  False
                235.25             0.000036  0.035536   False  False
                235.50             0.000056  0.056051   False  False
                235.75             0.000061  0.060840   False  False
                236.00             0.000061  0.060838   False  False
...                                     ...       ...     ...    ...
1980-01-31 49.5 292.00             0.000294  0.294192   False  False
                292.25             0.000437  0.436697   False  False
                292.50             0.000592  0.591902   False  False
                292.75             0.000599  0.598956   False  False
                293.00             0.000648  0.647638   False  False

[715077 rows x 4 columns]